<a href="https://colab.research.google.com/github/wldopie/ESAA/blob/main/HW_221118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **9.8 파이썬 추천 시스템 패키지 - Surprise**


---



### **9.8.1 Surprise 패키지 소개**
  - Surprise
    - 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 중 하나

  - Surprise 패키지의 주요 장점
    - 다양한 추천 알고리즘, 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템을 구축할 수 있다.
    - Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성되었다.

---

### **9.8.2 Surprise를 이용한 추천 시스템 구축**
  - 추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤, SVD 행렬 분해를 통한 잠재 요인 협업 필터링을 수행한다.

In [1]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 6.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633966 sha256=c2fa8cc48cbb417029f7c98e9661e86156a87942ffe79def14d5f97a9890c17a
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

  - `Surprise`는 Movies Lens 데이터 세트의 사용자-영화 평점 데이터 포맷과 같이 userID, movieID, rating과 같은 `주요 데이터가 Row 레벨 형태로 돼있는 포멧의 데이터만 처리`한다.

  - Surprise는 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트를 가져오는 API를 제공한다.

  - `Surprise Dataset 클래스`의 `load_builtin()`은 무비렌즈 사이트에서 제공하는 과거 버전 데이터 세트인 'ml-100k' 또는 'ml-1m' 데이터를 아카이브 사이트로부터 내려받아 로컬 디렉토리에 저장한 뒤 데이터를 로딩한다.

In [3]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


  - Surprise에서 사용하는 'ml-100k' 데이터 세트는 앞 예제에서 지금까지 사용한 movies.csv, ratings.csv 파일과는 차이가 있다.
  - 직접 내려받은 `movies.csv`, `ratings.csv`는 `최근 영화에 대한 평점 정보`를 가지고 있지만, Surprise가 내려받은 `ml-100k`, `ml-1m`은 `과거 버전의 데이터 세트`이다. `최신 데이터 세트`인 ratings.csv의 경우 `CSV 파일`이지만, `과거 버전의 데이터` 파일은 분리 문자가 `탭(\t) 문자`이다.

  - Surprise에 사용자-아이템 평점 데이터를 적용할 때 주의할 점은 무비렌즈 사이트에서 내려받은 데이터 파일과 동일하고 로우 레벨의 사용자-아이템 평점 데이터를 그대로 적용해야 한다는 것이다. 
  - Surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로, 원본인 로우 레벨의 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 한다.

  - 이제 SVD로 잠재 요인 협업 필터링을 수행해보자. 

In [4]:
algo = SVD()
algo.fit(trainset)

  - 학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행한다.
  - `Surprise`에서 `추천을 예측`하는 메서드
    1. `test()`
      - `사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측`하는 메서드
      - 즉, 입력된 데이터 세트에 대해 추천 데이터 세트를 만들어줌
    2. `predict()`
      - `개별 사용자와 영화에 대한 추천 평점`을 반환

In [5]:
# test()를 활용한 추천 예측
predictions = algo.test( testset )
print('prediction type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.3820728133375066, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.87654957154176, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.0549332793627615, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.7894127593906255, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.628442242489633, details={'was_impossible': False})]

  - SVD 알고리즘 객체의 test(데이터 세트) 메서드의 호출 결과는 파이썬 리스트이며, 크기는 입력 인자 데이터 세트의 크기와 같은 25000개이다.
  - 호출 결과로 반환된 리스트 객체는 25000개의 Prediction 객체를 내부에 가지고 있다.
    - `Prediction `객체는 `Surprise 패키지에서 제공하는 데이터 타입`이며, 아래의 데이터들을 `튜플 형태`로 가지고 있다.
      - uid : 개별 사용자 아이디
      - iid : 영화 아이디
      - r_ui : 실제 평점 정보
      - details : 내부 처리 시 추천 예측을 할 수 없는 경우 로그용으로 데이터를 남기는 데 사용되며, 'was_impossible'이 True이면 예측값을 생성할 수 없는 데이터라는 의미

  - 리스트 객체 내에 내포된 Prediction 객체의 uid, iid, r_ui, est 등의 속성에 접근하려면 객체명.uid와 같은 형식으로 가능하다.

In [6]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]] 

[('120', '282', 3.3820728133375066),
 ('882', '291', 3.87654957154176),
 ('535', '507', 4.0549332793627615)]

  - 이번에는 `Surprise 패키지`의 다른 추천 예측 메서드인 `predict()`를 이용해 `추천 예측`을 해보자.
  - predict()는 개별 사용자의 아이템에 대한 추천 평점을 예측해 준다.
  - 인자로 `개별 사용자 아이디`, `아이템 아이디를 입력`하면 `추천 예측 평점을 포함한 정보`를 반환한다.

In [7]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.32   {'was_impossible': False}


  - 이처럼 `predict()`는 `개별 사용자와 아이템 정보`를 입력하면 `추천 예측 평점을 est로 반환`한다.
  - `test()` 메서드는 `입력 데이터 세트의 모든 사용자와 아이템`에 대해서 `predict()를 반복적으로 수행`한 결과와 같다.

  - 테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이를 평가해 보자.
  - `Surprise의 accuracy` 모듈은 RMSE, MSE 등의 방법으로 `추천 시스템의 성능 평가 정보를 제공`한다. 

In [8]:
accuracy.rmse(predictions)

RMSE: 0.9478


0.9477654838166572

---

### **9.8.3 Surprise 주요 모듈 소개**

#### [Dataset]
  - Surprise는 user_id, item_id, rating 데이터가 로우 레벨로 된 데이터 세트만 적용할 수 있다.
  - Surprise의 API 명
    1. `Dataset.load_builtin()` : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터를 내려받는다.
    2. `Dataset.load_from_file(file_path, reader)` : OS 파일에서 데이터를 로딩할 때 사용한다.
    3. `Dataset.load_from_df(df, reader)` : 판다스의 DataFrame에서 데이터를 로딩한다.


#### [OS 파일 데이터를 Surprise 데이터 세트로 로딩]
  - Dataset.load_from_file API를 이용해 지정된 디렉터리에 있는 사용자-아이템 평점 데이터를 로딩해보자.
  - 먼저 Surprise에 OS 파일을 로딩할 때 주의할 점은 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있으면 안 된다는 것이다.
  - 따라서 판다스 DataFrame의 to_csv() 함수를 이용해 ratings.csv 파일의 헤더를 삭제하고 새로운 파일인 ratings_noh.csv로 저장한다.

In [9]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
# ratings_noh.csv 파일로 업로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('ratings_noh.csv', index = False, header = False)

  - 이제 ratings_noh.csv를 DataSet 모듈의 load_from_file()을 이용해 DataSet으로 로드해보자.
  - 먼저 Dataset.load_from_file()을 적용하기 전에 `Reader 클래스`를 이용해 `데이터 파일의 파싱 포맷을 정의`해야 한다.
  - 지금 로딩하려는 `ratings_noh.csv`는 `칼럼 헤더가 없고`, `4개의 칼럼이 콤마로만 분리`돼 있다. 이 4개의 칼럼이 사용자 아이디, 아이템 아이디, 평점, 타임스탬프임을 로딩할 때 알려줘야 한다.

In [10]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
data = Dataset.load_from_file('ratings_noh.csv', reader = reader)

  - `Reader 클래스`의 주요 생성 `파라미터`
    - `line_format` : 칼럼을 순서대로 나열, `입력된 문자열을 공백으로 분리`해 칼럼으로 인식
    - `sep` : 칼럼을 분리하는 `분리자`, 디폴트는 \t
    - `rating_scale` : `평점 값의 최소 ~ 최대` 평점을 설정, 디폴트는 (1, 5)

  - 이제 SVD 행렬 분해 기법을 이용해 추천을 예측해 보자.

In [11]:
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors = 50, random_state = 0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### [판다스 DataFrame에서 Surprise 데이터 세트로 로딩]
  - 판다스의 DataFrame에서 Surprise 데이터 세트로 로딩할 때 주의할 점은 DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 칼럼 순서를 지켜야 한다는 것이다.

In [12]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

algo = SVD(n_factors = 50, random_state = 0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

---

### **9.8.4 Surprise 추천 알고리즘 클래스**
  - `Surprise`에서 추천 예측을 위해 자주 사용되는 `추천 알고리즘 클래스`
    1. `SVD` : 행렬 분해를 통핸 잠재 요인 협업 필터링을 위한 SVD 알고리즘
    2. `KNNBasic` : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
    3. `BaselineOnly` : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

  - SVD 클래스의 입력 파라미터
    - n_factors : 잠재 요인 K의 개수. 커질수록 정확도가 높아질 수 있으나, 과적합 문제가 발생할 수 있음
    - n_epochs : SGD(확률적 경사하강법) 수행 시 반복 횟수
    - biased : 베이스라인 사용자 편향 적용 여부 



---

### **9.8.5 베이스라인 평점**
  - `베이스라인 평점`(Baseline Rating)
    - `개인의 성향을 반영`해 아이템 평가에 `편향성(bias) 요소를 반영`하여 `평점을 부과`하는 것

  - `베이스라인 평점`은 `전체 평균 평점` + `사용자 편향 점수` + `아이템 편향 점수` 공식으로 계산된다.
    - 전체 평균 평점 : 모든 사용자의 아이템에 대한 평점을 평균한 값
    - 사용자 편향 점수 : 사용자별 아이템 평점 평균 값 - 전체 평균 평점
    - 아이템 편향 점수 : 아이템별 평점 평균 값 - 전체 평균 평점

---

### **9.8.6 교차 검증과 하이퍼 파라미터 튜닝**
  - `Surprise`는 `교차 검증`과 `하이퍼 파라미터 튜닝`을 위해 사이킷런과 유사한 `cross_validate()`와 `GridSearchCV 클래스`를 제공한다.

In [13]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('ratings.csv')    # reading data in pandas df
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state = 0)
cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8755  0.8780  0.8680  0.8710  0.8685  0.8722  0.0039  
MAE (testset)     0.6737  0.6741  0.6688  0.6688  0.6676  0.6706  0.0027  
Fit time          6.39    5.13    5.20    5.11    5.21    5.41    0.49    
Test time         0.15    0.15    0.14    0.20    0.15    0.16    0.02    


{'test_rmse': array([0.87550506, 0.87796888, 0.86799422, 0.87099571, 0.86852108]),
 'test_mae': array([0.67367264, 0.67413459, 0.66880902, 0.66879766, 0.66758562]),
 'fit_time': (6.3927130699157715,
  5.129431247711182,
  5.198467254638672,
  5.106042861938477,
  5.206355571746826),
 'test_time': (0.15335726737976074,
  0.15206027030944824,
  0.14131641387939453,
  0.19565606117248535,
  0.14890599250793457)}

  - cross_validate()는 위의 출력 결과와 같이 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여준다.

  - `Surprise의 GridSearchCV`도 사이킷런의 GridSearchCV와 유사하게 `교차 검증을 통한 하이퍼 파라미터 최적화`를 수행한다.
  - `SVD`의 경우 주로 `SGD의 반복 횟수`를 지정하는 `n_epochs`와 `SVD의 잠재 요인 K의 크기`를 지정하는 `n_factors`를 튜닝한다.

In [14]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs' : [20, 40, 60], 'n_factors' : [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mae로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8767605383916356
{'n_epochs': 20, 'n_factors': 50}


  - 'n_epochs' : , 'n_factors' : 일 때 3개 폴드의 검증 데이터 세트에서 최적 RMSE가 로 도출되었다.

---

### **9.8.7 Surprise를 이용한 개인화 영화 추천 시스템 구축**
  - Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현해 보자.
  - `Surprise` 패키지로 `학습된 추천 알고리즘을 기반`으로 특정 사용자가 `아직 평점을 매기지 않은 영화 중`에서 `개인 취향에 가장 적절한 영화를 추천`해 보자.
  - 이번 예제에서는 ratings.csv 데이터를 학습 데이터와 테스트 데이터로 분리하지 않고 전체를 학습 데이터로 사용한다.
    - 그런데 Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수 없다.

In [16]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생한다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors = 50, random_state = 0)
algo.fit(data)

AttributeError: ignored

  - `데이터 세트 전체를 학습 데이터`로 사용하려면, `DatasetAutoFolds` 클래스를 이용하면 된다.
  - `DatasetAutoFolds` 객체를 생성한 뒤, `build_full_trainset()` 메서드를 호출하면 `전체 데이터를 학습 데이터 세트`로 만들 수 있다.

In [17]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file = 'ratings_noh.csv', reader = reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

  - DatasetAutoFolds의 build_full_trainset() 메서드를 이용해 생성된 학습 데이터를 기반으로 학습을 수행한 뒤, 특정 사용자에 영화를 추천하기 위해 아직 보지 않은 영화 목록을 확인해 보자.

In [18]:
algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
algo.fit(trainset)

  - 특정 사용자는 userId = 9인 사용자로 지정한다.
  - userId 9가 아직 평점을 매기지 않은 영화를 movieId 42로 선정한 뒤, 예측 평점을 계산해보자.

In [19]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('movies.csv')

# userId = 9의 movieId 데이터를 추출해 movieId = 42 데이터가 있는지 확인
movieIds = ratings[ratings['userId'] == 9]['movieId']

if movieIds[movieIds == 42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


  - `movieId 42`인 영화에 대해서 `userId 9` 사용자의 추천 `예상 평점`은 `predict()` 메서드를 이용하면 된다.
  - 학습된 `SVD 객체`에서 `predict()` 메서드 내에 `userId와 movieId 값을 입력`해주면 된다. 단, 이 값은 `모두 문자열` 값이어야 한다.

In [20]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


  - 추천 예측 평점은 3.13이다.

  - 이제 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤, 예측 평점 순으로 영화를 추천해 보자.

In [21]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수 :', len(seen_movies), '추천 대상 영화 수 :', len(unseen_movies),
        '전체 영화 수 :', len(total_movies))
  
  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수 : 46 추천 대상 영화 수 : 9696 전체 영화 수 : 9742


  - 9번 사용자는 전체 9742개 중 46개의 영화에만 평점을 매겼으므로, 추천 대상 영화는 9696개이다. 
  - 이 중 앞에서 학습된 추천 알고리즘 클래스인 SVD를 이용해 높은 예측 평점을 가진 순으로 영화를 추천해 보자.
  - `recomm_movie_by_surprise()` 함수는 `인자`로 `학습이 완료된 추천 알고리즘 객체`, `추천 대상 사용자 아이디`, `추천 대상 영화의 리스트 객체`, `추천 상위 N개 개수`를 받는다.

In [23]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 10):
  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
  # [Predictions(uid = '9', iid = '1', est = 3.69), Prediction(uid = '9', iid = '2', est = 2.98),,,]

  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key = sortkey_est, reverse = True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [ int(pred.iid) for pred in top_predictions ]
  top_movie_rating = [ pred.est for pred in top_predictions ]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [ (id, title, rating) for id, title, rating in
                     zip(top_movie_ids, top_movie_titles, top_movie_rating) ]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n = 10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수 : 46 추천 대상 영화 수 : 9696 전체 영화 수 : 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


  - 이처럼 Surprise 패키지는 복잡한 알고리즘을 직접 구현하지 않고도 쉽고 간결한 API를 이용해 파이썬 기반에서 추천 시스템을 구축할 수 있도록 한다.